<a href="https://colab.research.google.com/github/SalarShafiee/Image-classification/blob/main/Tomato-With%20TF.Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install -q kaggle

In [3]:
from google.colab import files

In [4]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"salarshafiee71","key":"702b6e7e2080f71cd323658809f86ee7"}'}

In [5]:
! mkdir ~/.kaggle

In [6]:
!kaggle datasets download -d noulam/tomato -p /content

Dataset URL: https://www.kaggle.com/datasets/noulam/tomato
License(s): copyright-authors
 98% 731M/743M [00:08<00:00, 75.0MB/s]
100% 743M/743M [00:08<00:00, 89.3MB/s]


In [7]:
!unzip -q /content/tomato.zip -d /content/tomato

In [8]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [9]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model  # Corrected 'model' to 'Model'
# from tensorflow.keras.applications.inception_v3 import InceptionV3  # Example commented out
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob  # Corrected a typo: 'glob.' to 'glob'

In [10]:
IMG_SIZE=[224,224]
train_path='/content/tomato/New Plant Diseases Dataset(Augmented)/train'
valid_path='/content/tomato/New Plant Diseases Dataset(Augmented)/valid'
BATCH_SIZE=16

In [11]:
import tensorflow as tf

In [12]:
train_dataset = tf.keras.utils.image_dataset_from_directory(train_path,
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE)

Found 18345 files belonging to 10 classes.


In [13]:
Valid_dataset = tf.keras.utils.image_dataset_from_directory(valid_path,
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE)

Found 4585 files belonging to 10 classes.


In [14]:
class_names = train_dataset.class_names
class_names

['Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spider_mites Two-spotted_spider_mite',
 'Tomato___Target_Spot',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 'Tomato___Tomato_mosaic_virus',
 'Tomato___healthy']

As the original dataset doesn't contain a test set, you will create one. To do so, determine how many batches of data are available in the validation set using tf.data.experimental.cardinality, then move 20% of them to a test set.

In [15]:
val_batches = tf.data.experimental.cardinality(Valid_dataset)
test_dataset = Valid_dataset.take(val_batches // 5)
Valid_dataset = Valid_dataset.skip(val_batches // 5)

In [16]:
print('Number of validation batches: %d' % tf.data.experimental.cardinality(Valid_dataset))
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))

Number of validation batches: 230
Number of test batches: 57


Use buffered prefetching to load images from disk without having I/O become blocking. To learn more about this method see the data performance guide.

In [17]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
Valid_dataset = Valid_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

In [18]:

data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

In [19]:
preprocess_input = tf.keras.applications.vgg16.preprocess_input

In [20]:
# Create the base model from the pre-trained model VGG16
IMG_SHAPE = IMG_SIZE + [3]
base_model = tf.keras.applications.VGG16(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

58889256/58889256 [==============================] - 0s 0us/step


In [21]:
base_model.trainable = False

Important note about BatchNormalization layers
Many models contain tf.keras.layers.BatchNormalization layers. This layer is a special case and precautions should be taken in the context of fine-tuning, as shown later in this tutorial.

When you set layer.trainable = False, the BatchNormalization layer will run in inference mode, and will not update its mean and variance statistics.

When you unfreeze a model that contains BatchNormalization layers in order to do fine-tuning, you should keep the BatchNormalization layers in inference mode by passing training = False when calling the base model. Otherwise, the updates applied to the non-trainable weights will destroy what the model has learned.

For more details, see the Transfer learning guide.

In [22]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [23]:
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

(16, 7, 7, 512)


In [24]:

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

(16, 512)


In [25]:
prediction_layer = tf.keras.layers.Dense(10)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)


(16, 10)


In [26]:
inputs = tf.keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [27]:

base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0, name='accuracy')])

In [ ]:

initial_epochs = 10

loss0, accuracy0 = model.evaluate(Valid_dataset)

153/230 [==================>...........] - ETA: 12:28 - loss: 4.8973 - accuracy: 0.1045

In [ ]:
VGG16=VGG16(input_shape=image_size+[3],weights='imagenet',include_top=False)

58889256/58889256 [==============================] - 4s 0us/step


In [ ]:
for layers in VGG16.layers:
  layers.trainable=False

In [ ]:
folders=glob('/content/tomato/New Plant Diseases Dataset(Augmented)/train/*')
folders

['/content/tomato/New Plant Diseases Dataset(Augmented)/train/Tomato___Target_Spot',
 '/content/tomato/New Plant Diseases Dataset(Augmented)/train/Tomato___Leaf_Mold',
 '/content/tomato/New Plant Diseases Dataset(Augmented)/train/Tomato___Tomato_mosaic_virus',
 '/content/tomato/New Plant Diseases Dataset(Augmented)/train/Tomato___Bacterial_spot',
 '/content/tomato/New Plant Diseases Dataset(Augmented)/train/Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 '/content/tomato/New Plant Diseases Dataset(Augmented)/train/Tomato___Spider_mites Two-spotted_spider_mite',
 '/content/tomato/New Plant Diseases Dataset(Augmented)/train/Tomato___Early_blight',
 '/content/tomato/New Plant Diseases Dataset(Augmented)/train/Tomato___Late_blight',
 '/content/tomato/New Plant Diseases Dataset(Augmented)/train/Tomato___healthy',
 '/content/tomato/New Plant Diseases Dataset(Augmented)/train/Tomato___Septoria_leaf_spot']

In [ ]:
x=Flatten()(VGG16.output)
prediction=Dense(len(folders),activation='softmax')(x)

In [ ]:
model=Model(inputs=VGG16.input, outputs=prediction)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
model.compile(optimizer='adam',metrics=['accuracy'],loss='categorical_crossentropy')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

In [ ]:
training_set=train_datagen.flow_from_directory(train_path,target_size=(224,224),batch_size=16,class_mode='categorical')

Found 18345 images belonging to 10 classes.


In [ ]:
test_set=train_datagen.flow_from_directory(valid_path,target_size=(224,224),batch_size=16,class_mode='categorical')

Found 4585 images belonging to 10 classes.


In [ ]:
history=model.fit_generator(training_set,validation_data=test_set,epochs=10)

<ipython-input-29-2f334b29a5f8>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history=model.fit_generator(training_set,validation_data=test_set,epochs=10)


Epoch 1/10
1147/1147 [==============================] - 343s 293ms/step - loss: 0.8094 - accuracy: 0.7469 - val_loss: 0.6599 - val_accuracy: 0.7952
Epoch 2/10
1147/1147 [==============================] - 335s 292ms/step - loss: 0.5130 - accuracy: 0.8438 - val_loss: 0.4506 - val_accuracy: 0.8595
Epoch 3/10
1147/1147 [==============================] - 317s 277ms/step - loss: 0.4802 - accuracy: 0.8597 - val_loss: 0.5597 - val_accuracy: 0.8449
Epoch 4/10
1147/1147 [==============================] - 312s 272ms/step - loss: 0.3880 - accuracy: 0.8878 - val_loss: 0.4344 - val_accuracy: 0.8820
Epoch 5/10
1147/1147 [==============================] - 313s 273ms/step - loss: 0.3730 - accuracy: 0.8945 - val_loss: 0.6064 - val_accuracy: 0.8526
Epoch 6/10
1147/1147 [==============================] - 311s 271ms/step - loss: 0.3387 - accuracy: 0.9078 - val_loss: 0.5092 - val_accuracy: 0.8739
Epoch 7/10
1147/1147 [==============================] - 331s 288ms/step - loss: 0.3394 - accuracy: 0.9079 - val_